In [1]:
from lambeq import BobcatParser, TreeReader, TreeReaderMode, spiders_reader, cups_reader, stairs_reader
from lambeq import TensorAnsatz, SpiderAnsatz, MPSAnsatz, AtomicType
from discopy import Dim
from classic_pipeline import *
from utilities import *

In [2]:
#define atomic-types

N = AtomicType.NOUN
S = AtomicType.SENTENCE
C = AtomicType.CONJUNCTION
P = AtomicType.PUNCTUATION
NP = AtomicType.NOUN_PHRASE
PP = AtomicType.PREPOSITIONAL_PHRASE

In [3]:
#parser declaration

bobcat_parser = BobcatParser(verbose = "progress")
spider_parser = spiders_reader
cups_parser = cups_reader
stairs_parser = stairs_reader
tree_parser = TreeReader(mode=TreeReaderMode.RULE_ONLY)

In [4]:
#ansatze declaration

tensor_ansatz = TensorAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)})
spider_ansatz = SpiderAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)})
mps_ansatz = MPSAnsatz({N: Dim(2), S: Dim(2), C: Dim(2), P: Dim(2), NP: Dim(2), PP: Dim(2)}, bond_dim = 3)

In [ ]:
#data-extraction for classic pipeline (linux)

pip = ClassicPipeline(cups_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
train_labels, train_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/GPS_edited.csv", "n")
test_labels, test_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/CPN_edited.csv", "n")
eval_labels, eval_circuits = pip.create_circuits_and_labels("/home/adriano22_/Documents/GitHub/Tesi-Quantum-NLP/project/datasets/edited_datasets/ePurse_edited.csv", "n")

In [5]:
#data-extraction for classic pipeline (win11)

pip = ClassicPipeline(cups_parser, tensor_ansatz)
pip.add_rewriter_rules(ClassicPipeline.SUPPORTED_RULES[0], ClassicPipeline.SUPPORTED_RULES[1], ClassicPipeline.SUPPORTED_RULES[4])
#train_labels, train_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\GPS_edited.csv", "n")
#test_labels, test_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\CPN_edited.csv", "n")
#eval_labels, eval_circuits = pip.create_circuits_and_labels("C:\\Users\\calif\\Documents\\GitHub\\Tesi-Quantum-NLP\\project\\datasets\\edited_datasets\\ePurse_edited.csv", "n")

In [6]:
#save_data("train_data.txt", train_labels, train_circuits)
#save_data("test_data.txt", test_labels, test_circuits)
#save_data("eval_data.txt", eval_labels, eval_circuits)

train_labels, train_circuits = load_data("train_data.txt")
test_labels, test_circuits = load_data("test_data.txt")
eval_labels, eval_circuits = load_data("eval_data.txt")

In [9]:
#train-set (GPS) faulty entries: 127:128 128:129 129:130 136:137 137:138 138:139 147:148 150:151 151:152 155:156 156:157
#test-set (CPN) faulty entries: None
#eval-set (ePurse) faulty entries: 22:23 34:46


rtrain_circuits = train_circuits[0:127] + train_circuits[130:136] + train_circuits[139:147] + train_circuits[148:150] + train_circuits[152:155] + train_circuits[157:]
reval_circuits = eval_circuits[0:22] + eval_circuits[23:34] + eval_circuits[46:]


In [ ]:
"""
working parser/ansatz combos:
    - cups / (tensor, spider, mps) : works on all requirements    
    - stairs / (tensor) : works on all requirements
    - tree / (tensor) : works on all requirements
    - bobcat / (tensor, spider, mps) : works with 90+% of requirements
    
NB: missing combos don't work together, common exception raised is AxiomError   
"""

In [7]:
#training block for classical pipeline

train_set, test_set, eval_set = pip.create_dataset(train_labels, train_circuits), pip.create_dataset(test_labels, test_circuits), pip.create_dataset(eval_labels, eval_circuits)
model = pip.create_model(train_circuits, test_circuits, eval_circuits)
trainer = pip.create_trainer(model = model)
#pip.create_trainer(model = model, loss = torch.nn.HingeEmbeddingLoss(), optimizer = torch.optim.Adam, n_epochs = 10)
#pip.train_model(train_set, eval_set)
#pip.plot() 

In [ ]:
epochs = [100]
loss_functions = [torch.nn.BCEWithLogitsLoss(), torch.nn.HingeEmbeddingLoss()]
optimizers = [torch.optim.AdamW, torch.optim.Adagrad, torch.optim.Adam, torch.optim.Adamax]
learning_rates = [3e-1, 3e-2, 3e-3]

pip.create_trainer(model = model, loss = loss_functions[1], optimizer = optimizers[3], n_epochs = 100, lr = learning_rates[2])
pip.train_model(train_set, eval_set, 1, 1)


"""#path = "C:\\Users\\calif\\Desktop\\grid_results\\"



for loss in loss_functions:
    for optimizer in optimizers:
        for lr in learning_rates:
            print(f"\nCurrent triple: \n -LEARNING RATE: {str(lr)}\n -LOSS FUNCTION: {str(loss)}\n -OPTIMIZER: {str(optimizer)}")
            pip.create_trainer(model = model, loss = loss, optimizer = optimizer, n_epochs = 100, lr = lr)
            pip.train_model(train_set, eval_set, 1, 1)
            #filename = path + str(lr) + "_" + str(loss) + "_" + str(optimizer)
            #pip.plot(filename)"""

Posso rimuovere Adadelta optimizer dalla gridsearcg perchè non dà miglioramenti al training.
Quando si usa HingeEmbeddedLoss ottengo loss value tendenti -infinito.



Dal primo round di gridsearch:
    - con la loss function HingeEmbeddedLoss ottengo sempre valori negativi con conseguenti score bassi nelle metriche di valutazione.
    - con la loss function BCEWithLogitsLoss i valori sono mediamente stabili, le epoch piu equilibrate individuate sono:
            
            Epoch 80:   train/loss: 0.1989   valid/loss: 4.3054   train/acc: 0.8720   train/rec: 0.8571   train/f1: 0.8701   valid/acc: 0.5202   valid/rec: 0.5403   valid/f1: 0.5296     -LEARNING RATE: 0.03  -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adamw.AdamW'>

            Epoch 90:   train/loss: 0.3694   valid/loss: 6.0657   train/acc: 0.7649   train/rec: 0.7381   train/f1: 0.7584   valid/acc: 0.5242   valid/rec: 0.4919   valid/f1: 0.5083     -LEARNING RATE: 0.003 -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adamw.AdamW'>

            Epoch 95:   train/loss: 0.6820   valid/loss: 0.7440   train/acc: 0.5327   train/rec: 0.4583   train/f1: 0.4952   valid/acc: 0.5000   valid/rec: 0.4435   valid/f1: 0.4701     -LEARNING RATE: 0.003 -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adagrad.Adagrad'>

            Epoch 71:   train/loss: 0.2060   valid/loss: 1.2076   train/acc: 0.8571   train/rec: 0.8452   train/f1: 0.8554   valid/acc: 0.5363   valid/rec: 0.5000   valid/f1: 0.5188     -LEARNING RATE: 0.03 -LOSS FUNCTION: BCEWithLogitsLoss() -OPTIMIZER: <'torch.optim.adam.Adam'>


    Le restanti combinazioni non sono riportate perchè hanno o loss troppo alte oppure score delle metriche troppo basse.





In [ ]:
from ray import tune
from ray.tune.tuner import Tuner, TuneConfig

parameters = {
    "params": {
        "epochs": [10, 20, 40, 60, 80, 100],
        "loss_function": [torch.nn.BCELoss, torch.nn.BCEWithLogitsLoss, torch.nn.HingeEmbeddingLoss],
        "optimizer": [torch.optim.AdamW, torch.optim.Adadelta, torch.optim.Adagrad, torch.optim.Adam, torch.optim.Adamax]  
    }      
}

tuner = Tuner(trainer, param_space = parameters, tune_config = TuneConfig(num_samples = 5, metric = "train-logloss", mode = "min"))
result_grid = tuner.fit()
best = result_grid.get_best_result()
print(best)


In [ ]:
from sklearn.model_selection import *
from sklearn.metrics import *

trainer = pip.create_trainer(model = model, loss = torch.nn.BCEWithLogitsLoss, n_epochs = 10)
#l_estimator = LambeqEstimator(trainer)


parameters = {
    "epochs": [10, 20, 40, 60, 80, 100],
    "loss_function": [torch.nn.BCELoss, torch.nn.BCEWithLogitsLoss, torch.nn.HingeEmbeddingLoss],
    "optimizer": [torch.optim.AdamW, torch.optim.Adadelta, torch.optim.Adagrad, torch.optim.Adam, torch.optim.Adamax]    
}

scores = ["accuracy", "recall", "f1"]

tuner = GridSearchCV(estimator = trainer, param_grid = parameters, scoring = "f1", n_jobs = -1, cv = 5)
tuner.fit(train_circuits, train_labels)